## 1 FOR THE DEMO run these first

In [ ]:
!pip install tensorflow[and-cuda]

In [ ]:
!pip install SpeechRecognition
!pip install google-cloud-texttospeech

In [ ]:
# Install Gradio
!pip install gradio


In [1]:
import kagglehub
kagglehub.login()

## 2 FOR THE DEMO run this first to load the Fine-Tuned model

In [2]:
import kagglehub

# Download latest version
path = kagglehub.model_download("joelbest/gemma2b_reo/keras/default")

print("Path to model files:", path)

Path to model files: /home/jupyter/.cache/kagglehub/models/joelbest/gemma2b_reo/keras/default/1


In [3]:
# run this first to load the finetuned model# import keras so it is defined for later use
import keras
import keras_nlp

# Load the saved model in .keras format
gemma_lm = keras.models.load_model('/home/jupyter/.cache/kagglehub/models/joelbest/gemma2b_reo/keras/default/1/my_model.keras')


2024-12-01 18:09:06.118103: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-01 18:09:06.154009: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733076546.178286    5217 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733076546.185571    5217 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-01 18:09:06.222221: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
# output to show the model is sucessfully loaded
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

## 3 FOR THE DEMO run this after you load the model

In [5]:
# FOR THE DEMO run this after you load the model
import datetime
import gradio as gr
import tensorflow as tfG
import keras
import speech_recognition as sr
from google.cloud import texttospeech

#GW: I moved the imports to the cell above

# Initialize the Text-to-Speech client
client = texttospeech.TextToSpeechClient()

# Initialize the Speech Recognition
recognizer = sr.Recognizer()

# Function to process Speech-to-Text
def speech_to_text(audio):
    try:
        print("sr.AudioFile start: ", datetime.datetime.now(), "audio=", type(audio), audio)
        with sr.AudioFile(audio) as source:
            print("sr.AudioFile start2: ", datetime.datetime.now(), "source=", type(source))
            audio_data = recognizer.record(source)
            text = recognizer.recognize_google(audio_data)
            print("sr.AudioFile end: ", datetime.datetime.now())
            return text
    except sr.UnknownValueError:
        return "Sorry, I couldn't understand the audio. Please try again."
    except sr.RequestError as e:
        return f"Request error from Google STT service; {e}"

# Function to generate response from the chatbot model
def generate_response(model: keras.Model, question: str, max_length: int = 250) -> str:
    prompt_template = """ Role: You will use layman english to explain yourself. You will explain everything in simple terms using layman english.
    You are a helpful real estate assistant providing support to legal matters in Ontario, 
    Canada Region; always answer the questions based on the following instructions; given the question, 
    generate an answer in 5 to 6 sentences always. For every statement in the answer, provide supporting 
    legal document or law segments from Ontario, Canada region.
    Question:\n{question}\n\nAnswer:\n"""
    
    prompt = prompt_template.format(question=question)
    print("generate_response start: ", datetime.datetime.now(), "prompt-->", prompt,"<--")
    
    try:
        generated_response = model.generate(prompt, max_length=max_length)
        generated_text = generated_response.decode('utf-8') if isinstance(generated_response, bytes) else generated_response
        answer_text = generated_text[len(prompt):].strip()
        print("generate_response generated: ", datetime.datetime.now(), "answer text-->", answer_text, "<--")
        return answer_text
    except Exception as e:
        return f"⚠️ Error generating answer: {e}"

# Function to synthesize text using Google Text-to-Speech
def synthesize_text(text):
    input_text = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        name="en-US-Standard-C",
        ssml_gender=texttospeech.SsmlVoiceGender.FEMALE
    )
    audio_config = texttospeech.AudioConfig(audio_encoding=texttospeech.AudioEncoding.MP3)
    
    print("client synthesize_speech start: ", datetime.datetime.now())
    response = client.synthesize_speech(
        request={"input": input_text, "voice": voice, "audio_config": audio_config}
    )
    print("client synthesize_speech finished: ", datetime.datetime.now())
    audio_file_path = "output.mp3"
    with open(audio_file_path, "wb") as out:
        out.write(response.audio_content)
    return audio_file_path

# Combined function to handle chatbot response and TTS
def chatbot_with_tts_and_stt(audio):
    print("chatbot start: ", datetime.datetime.now())
    # Convert speech to text
    user_input = speech_to_text(audio)
    if not user_input.strip():
        return "Chatbot: I'm here to help! Please try speaking again.", None
    
    # Confirm the input text
    confirmation_message = f"Recognized text: '{user_input}'. Is this correct? If so, proceed. If not, try again."
    # Optionally, you can add a confirmation mechanism here
    
    # Generate chatbot response
    answer = generate_response(gemma_lm, user_input, max_length=250)
    
    # Synthesize the generated text into speech
    audio_file_path = synthesize_text(answer)
    
    print("chatbot finish: ", datetime.datetime.now())
    return answer, audio_file_path

# Create Gradio interface THIS LOOKS GOOD NEED REMOVE WATERMARK CHARACTER
interface = gr.Interface(
    fn=chatbot_with_tts_and_stt,
    inputs=gr.Audio(sources=["microphone"], type="filepath"),
    outputs=[gr.Textbox(), gr.Audio(autoplay=True)],
    title="""
    <div style="display: flex; flex-direction: column; align-items: center; text-align: center; margin-bottom: 10px;">
        <img src="http://getmuggle.com/wp-content/uploads/2024/11/Logo-shrunk.png" style="height: 180px; margin-bottom: -10px;">
        <h1 style="margin: 0; font-size: 24px;">Real Estate Mate</h1>
    </div>
    """,
    description="Speak your question and get a spoken answer.",
    # theme='Zarkel/IBM_Carbon_Theme'
    theme='NoCrypt/miku'
    # theme='shivi/calm_seafoam'
    # theme='gstaff/xkcd'
    # theme='NoCrypt/miku'
    # theme='Base'
)

# Launch the interface
demo = interface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2024/12/01 18:11:45 [W] [service.go:132] login to server failed: dial tcp 44.237.78.176:7000: connect: connection refused


## 4 Run this code to close the interface


In [ ]:
interface.close()

## 5 Run this code if you want to clean up memory / resources

In [ ]:
import gc
import tensorflow as tf
from numba import cuda

def get_gpu_memory_usage():
    """Get the current and peak GPU memory usage."""
    gpus = tf.config.list_physical_devices('GPU')
    if not gpus:
        return 0, 0  # No GPU available
    memory_info = tf.config.experimental.get_memory_info('GPU:0')
    return memory_info['current'], memory_info['peak']

# Get GPU memory usage before cleanup
before_memory, before_peak = get_gpu_memory_usage()

# Delete any variables holding models or large objects
try:
    del gemma_lm  # Delete the model if it exists
except NameError:
    pass  # If gemma_lm doesn't exist, continue

# Clear TensorFlow GPU memory
tf.keras.backend.clear_session()

# Run garbage collection to free memory
gc.collect()

# Reset GPU memory
try:
    device = cuda.get_current_device()
    device.reset()
except Exception as e:
    print(f"CUDA reset failed: {e}")

# Get GPU memory usage after cleanup
after_memory, after_peak = get_gpu_memory_usage()

# Calculate and display the memory cleared
cleared_memory = before_memory - after_memory
print(f"Memory before cleanup: {before_memory / 1024**2:.2f} MB")
print(f"Memory after cleanup: {after_memory / 1024**2:.2f} MB")
print(f"Memory cleared: {cleared_memory / 1024**2:.2f} MB")
